In [ ]:
! pip install selenium==3.141.0

In [ ]:
! pip install urllib3==1.26.11

In [ ]:
! pip install chardet==5.2.0

In [ ]:
! wget https://storage.googleapis.com/chrome-for-testing-public/133.0.6943.98/linux64/chromedriver-linux64.zip

In [ ]:
! unzip chromedriver-linux64.zip

In [ ]:
! sudo apt install -y libvulkan1

In [ ]:
! wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb

In [ ]:
! sudo dpkg -i google-chrome-stable_current_amd64.deb

In [ ]:
! ps -ef |grep chrome | awk '{print $2}' | xargs kill -9

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import os
import requests
import ssl
import urllib.request

os.environ['OPENSSL_CONF'] = '/usr/lib/ssl/openssl.cnf'

# Function to check if the webpage exists using requests
def is_page_available(url):
    try:
        ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        ctx.options |= 0x4
        response = urllib.request.urlopen(url, context=ctx)
        if response.status == 200:
            return True
        else:
            return False

    except Exception as e:
        # print(f"Page {url} does not exist")
        return False

url_prefix = 'DTXSID'
download_dir = '/home/ubuntu/toxcast_collect/bioactivities'
chromedriver_path = '/home/ubuntu/toxcast_collect/chromedriver-linux64/chromedriver'
dtxsids_file = 'DTXSIDs'

waiting_seconds = 10

# Set up Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration (recommended for headless)
chrome_options.add_argument("--no-sandbox")  # Prevents errors in certain environments (like Docker)

# Specify the download directory
prefs = {
    "download.default_directory": download_dir,  # Set download directory
    "download.prompt_for_download": False,        # Disable download prompt
    "directory_upgrade": True                     # Allow changing the download directory
}

chrome_options.add_experimental_option("prefs", prefs)

with open(dtxsids_file, 'r') as f:
    lines = f.readlines()

    for i, line in enumerate(lines):
        dtxsid = line.strip()
        url = f"https://comptox.epa.gov/dashboard/chemical/invitrodb/{dtxsid}"

        if is_page_available(url):
            print(f'page {url} is available')

            # Set up the WebDriver (Make sure you've downloaded the correct WebDriver for your browser)
            driver = webdriver.Chrome(executable_path=chromedriver_path, options=chrome_options)  # Provide your chromedriver path
            driver.set_window_size(1920, 1080)
            #driver.maximize_window()
            print(f'waiting for page loading {url}')
            driver.get(url)
            # Give the page some time to load
            time.sleep(waiting_seconds)  # You can adjust the sleep time depending on your network speed

            page_title = driver.title
            print(f'[{i}] {page_title} {url_prefix}{dtxsid}')

            # Locate the <span> element with exact text "Export data"
            # span_element = driver.find_elements(By.XPATH, "//span//div//span[text()='Export Data']")

            # span_element = driver.find_element(By.XPATH, "//span[text()='Export Data']")
            span_element = driver.find_element(By.ID, "bioactivitySummary")
            driver.execute_script(f"window.scrollTo(0, {span_element.location['y'] - 50});")

            actions_activate = ActionChains(driver)

            # Alternatively, using ActionChains (if needed)
            actions_activate.move_by_offset(1169, 20).click()
            actions_activate.move_by_offset(407, 0).click()
            actions_activate.move_by_offset(-328, 105).click()
            actions_activate.move_by_offset(33, 110).click()

            # actions_save = ActionChains(driver)
            actions_activate.move_by_offset(-1033, -225).click()
            # driver.save_screenshot('test3.png')
            actions_activate.move_by_offset(80, 45).click().perform()
            time.sleep(waiting_seconds)

            driver.quit()

    else:
        print(f'page {url} is not available')
